### NN Policy

In [2]:
env = gym.make("CartPole-v0")

In [2]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected

In [3]:
n_inputs = 4 
# == env.observation_space.shape[0]
n_hidden = 4 
# it's a simple task, we don't need more hidden neurons
n_outputs = 1 
# only outputs the probability of accelerating left

In [31]:
initializer = tf.contrib.layers.variance_scaling_initializer()

In [30]:
Image(url='https://blog.paperspace.com/content/images/2018/06/ELU.png', width=200)

In [35]:
X = tf.placeholder(
    dtype=tf.float32, 
    shape=[None, n_inputs],
    name=None
)

In [36]:
hidden = fully_connected(
    inputs=X, 
    num_outputs=n_hidden, 
    activation_fn=tf.nn.elu, 
    weights_initializer=initializer
)

In [37]:
logits = fully_connected(
    inputs=hidden, 
    num_outputs=n_outputs, 
    activation_fn=None,
    weights_initializer=initializer
)

In [39]:
outputs = tf.nn.sigmoid(
    x=logits
)

In [48]:
# select a random action based on the estimated probabilities
p_left_and_right = tf.concat(axis=1, values=[outputs, 1 - outputs])

action = tf.multinomial(tf.log(p_left_and_right), num_samples=1)

In [49]:
init = tf.global_variables_initializer()

In [54]:
# target is 1 if action was left (0) and 
y = 1. - tf.to_float(action)

In [55]:
learning_rate = 0.01

cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(
    labels=y, 
    logits=logits
)

optimizer = tf.train.AdamOptimizer(learning_rate)

In [56]:
grads_and_vars = optimizer.compute_gradients(cross_entropy)

In [57]:
gradients = [grad for grad, variable in grads_and_vars]

In [62]:
gradient_placeholders = []
grads_and_vars_feed = []
for grad, variable in grads_and_vars:
    gradient_placeholder = tf.placeholder(tf.float32, shape=grad.get_shape())
    gradient_placeholders.append(gradient_placeholder)
    grads_and_vars_feed.append((gradient_placeholder, variable))
    training_op = optimizer.apply_gradients(grads_and_vars_feed)

In [64]:
def discount_rewards(rewards, discount_rate):
    discounted_rewards = np.empty(len(rewards))
    cumulative_rewards = 0
    for step in reversed(range(len(rewards))):
        cumulative_rewards = rewards[step] + cumulative_rewards * discount_rate
        discounted_rewards[step] = cumulative_rewards
    
    return discounted_rewards

def discount_and_normalize_rewards(all_rewards, discount_rate):
    all_discounted_rewards = [discount_rewards(rewards) for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
        
    return [(discounted_rewards - reward_mean)/reward_std for discounted_rewards in all_discounted_rewards]